In [58]:
!pip install torchdiffeq

In [59]:
import torch
import torch.nn as nn
from torchdiffeq import odeint
import numpy as np
import pandas as pd
from scipy.interpolate import interp1d

In [60]:
#Data stuff happens here
time_res = 2000
t_fine = np.linspace(0,76,time_res)

url = 'http://people.whitman.edu/~hundledr/courses/M250F03/LynxHare.txt'
df = pd.read_csv(url, delim_whitespace=True, header=None, index_col=0)
df.index.name = 'Year'
df.columns = ['Hare', 'Lynx']

raw_years = df.index.to_numpy()  # [1845, ..., 1921]
shifted_years = raw_years - raw_years[0]  # [0, ..., 76

hares = df['Hare']
lynxes = df['Lynx']

hares_interp = interp1d(shifted_years, hares, kind='cubic')
lynxes_interp = interp1d(shifted_years, lynxes, kind='cubic')

hares_fine = hares_interp(t_fine)
lynxes_fine = lynxes_interp(t_fine)

data = np.array([hares_fine, lynxes_fine]).T

data_norm = (data - np.mean(data, axis=0)) / np.std(data, axis=0)

<ipython-input-60-b3198017a5b0>:6: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(url, delim_whitespace=True, header=None, index_col=0)


In [67]:
class LVParamLearner(nn.Module):
    def __init__(self):
      super().__init__()
      self.net = nn.Sequential(
            nn.Linear(2*(time_res), 8),
            nn.Tanh(),
            nn.Linear(8, 4) #4 for alpha, beta, gamma and delta
        )

    def forward(self, x):
      x = torch.flatten(torch.tensor(x,dtype=torch.float32))
      return self.net(x)

In [71]:
paramlearner = LVParamLearner()
optimizer = torch.optim.Adam(paramlearner.parameters(), lr=1e-4)

In [72]:
def make_deltas(alpha, beta, gamma, delta):
    def dynamics(t, y):
        return torch.stack([
            alpha * y[0] - beta * y[0] * y[1],
            delta * y[0] * y[1] - gamma * y[1]
        ])
    return dynamics

In [73]:
y0 = data[0]
y0 = torch.tensor(y0)
t_fine = torch.tensor(t_fine)
pred = 0
for epoch in range(250):
  optimizer.zero_grad()

  params = paramlearner(data_norm)

  alp,bet,gam,delt = nn.functional.softplus(params)
  dynamics = make_deltas(alp,bet,gam,delt)
  # deltas = torch.stack([alpha*data[0] - beta*data[0]*data[1],delta*data[0]*data[1]-gamma*data[1]])

  pred = odeint(dynamics,y0,t_fine,method='rk4')

  target = torch.tensor(data_norm)
  loss = torch.mean(((pred - target)**2))

  loss.backward()
  optimizer.step()

  print(f"EPoch: {epoch} and Loss:{loss}")


<ipython-input-73-b82d520a42cb>:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  t_fine = torch.tensor(t_fine)


EPoch: 0 and Loss:12.865527267612801
EPoch: 1 and Loss:10.248483056852615
EPoch: 2 and Loss:8.35873350759967
EPoch: 3 and Loss:7.167615758422247
EPoch: 4 and Loss:6.487041130517926
EPoch: 5 and Loss:6.129741568867155
EPoch: 6 and Loss:5.964298021396826
EPoch: 7 and Loss:5.873994145533908
EPoch: 8 and Loss:5.802855086431693
EPoch: 9 and Loss:5.74014904115012
EPoch: 10 and Loss:5.687704441328668
EPoch: 11 and Loss:5.651018804505705
EPoch: 12 and Loss:5.63133380679578
EPoch: 13 and Loss:5.622857350073271
EPoch: 14 and Loss:5.619201963884979
EPoch: 15 and Loss:5.616719962831043
EPoch: 16 and Loss:5.613788682168547
EPoch: 17 and Loss:5.609819874950152
EPoch: 18 and Loss:5.6046675396294
EPoch: 19 and Loss:5.598339790864825
EPoch: 20 and Loss:5.590906457353424
EPoch: 21 and Loss:5.582489551776922
EPoch: 22 and Loss:5.573313035100913
EPoch: 23 and Loss:5.563764302317908
EPoch: 24 and Loss:5.554449759542444
EPoch: 25 and Loss:5.546212008683689
EPoch: 26 and Loss:5.540006573996976
EPoch: 27 and 

KeyboardInterrupt: 

In [74]:
print(pred)
print(data_norm)
print(nn.functional.softplus(paramlearner(data)))

tensor([[1.9580e+01, 3.0090e+01],
        [6.3921e+00, 3.4763e+01],
        [1.9966e+00, 3.5213e+01],
        ...,
        [6.9626e-01, 2.7520e-38],
        [7.0572e-01, 2.6600e-38],
        [7.1532e-01, 2.5715e-38]], dtype=torch.float64, grad_fn=<CopySlices>)
[[-0.74064596  0.10569586]
 [-0.74747527  0.14331119]
 [-0.7535079   0.18037613]
 ...
 [ 0.63965921 -0.96838023]
 [ 0.64484963 -0.95128741]
 [ 0.65044458 -0.93391234]]
tensor([0.3785, 0.9234, 1.1821, 0.4060], grad_fn=<SoftplusBackward0>)
